In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
import collections
import nltk
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
import itertools
from collections import Counter

In [ ]:
import os
import glob
from bs4 import BeautifulSoup

# reading the share repurchase csv file

In [ ]:
share_repurchase =pd.read_csv('data/share_repurchase_paragraphs.csv')

In [ ]:
training_filings_list = []
for root, dirs, files in os.walk('data/nc_training_filings/'):
    training_filings_list += glob.glob(os.path.join(root, '*.html'))

In [ ]:
print(training_filings_list[0])

In [ ]:
with open(training_filings_list[0]) as file:
    raw1=file.read()

In [ ]:
raw1 = raw1.lower()

In [ ]:
type(raw1)

# using Beautiful soup to read the raw file

In [ ]:
html1 = BeautifulSoup(raw1)

In [ ]:
#tokens = [w for w in word_tokenize(file_html.lower()) if w.isalpha()]
#stop_words = set(stopwords.words("english"))
#text = nltk.Text([t for t in word_tokenize(raw1) if t not in stopwords.words('english')])

In [ ]:
type(raw1)

In [ ]:
len(raw1)

In [ ]:
raw1[:100]

In [ ]:
tag_removed = html1.body.text

In [ ]:
tokens = word_tokenize(tag_removed)

In [ ]:
type(tokens)

In [ ]:
len(tokens)

In [ ]:
tokens[:10]

In [ ]:
text = nltk.Text(tokens)

In [ ]:
len(text)

In [ ]:
type(text)

In [ ]:
text[:10]

In [ ]:
text[40:65]

In [ ]:
text.concordance('share',width=79,lines=25)

In [ ]:
text.concordance('authorization',width=100,lines=55)

In [ ]:
text.concordance('repurchase',width=79,lines=25)

In [ ]:
text.concordance('Authorization Date',width=100,lines=50)

In [ ]:
text.concordance('Intention',width=79,lines=25)

In [ ]:
text.collocations(num=10)

In [ ]:
wordlist = ['share','repurchase','authorization']

In [ ]:
wordlist1 = [w for w in wordlist]

In [ ]:
[w for w in text if re.search('(share|chase|zation)$', w)]
print('share  word appeared : ' , [len(w) for w in text if w=='share'])
print("------------")
print("repurchase word appeared :" , [len(w) for w in text if w=='repurchase'])
print("------------")
print("authorization word appeared  :" , [len(w) for w in text if w == 'authorization'])


In [ ]:
text.tokens.count('repurchase')

In [ ]:
text.tokens.count('authorization')

In [ ]:
share_repurchase.head()

In [ ]:
len(share_repurchase)

In [ ]:
share_repurchase.groupby('accession_number').size().head(5)

In [ ]:
dfkn_array = share_repurchase.data_key_friendly_name.unique()

In [ ]:
share_repurchase.data_key_friendly_name.value_counts()

In [ ]:
share_repurchase[share_repurchase.accession_number == '0000004127-17-000033'] #0000004127-17-000033

In [ ]:
dkfn_list = 'Share Repurchase Authorization Date Share Repurchase Authorization Amount Spent on Share Repurchases Share Repurchase Count Share Repurchase Intention Unknown Share Repurchase Data Share Repurchase Utilization'

In [ ]:
dfkn_tokens = nltk.word_tokenize(dkfn_list)

In [ ]:
dfkn_tokens[:10]

In [ ]:
text1 =share_repurchase[share_repurchase.data_key_friendly_name == 'Share Repurchase Authorization'].text
tokens =   [word_tokenize(t.lower()) for t in text1]

In [ ]:
type(list_tokens)

In [ ]:
len(list_tokens)

In [ ]:
list_tokens[:10]

In [ ]:
tokens[:10]

# flatten the list of lists 

In [ ]:
list_tokens1 = list(itertools.chain.from_iterable(tokens))


In [ ]:
type(list_tokens1)

In [ ]:
len(list_tokens1)

In [ ]:
list_tokens1[:10]

In [ ]:
stop_words = set(stopwords.words("english"))

In [ ]:
no_stopwords = [t for t in list_tokens1 if t not in stop_words]

In [ ]:
no_stopwords[:10]

In [ ]:
share_repurchase_text = Counter(no_stopwords)

In [ ]:
share_repurchase_text.most_common(10)

In [ ]:
bigram = nltk.bigrams(share_repurchase_text)

In [ ]:
list(bigram)

In [ ]:
trigram = nltk.trigrams(share_repurchase_text)

In [ ]:
list(trigram)

# using nltk collocation to score bigram - highest scoring pairM



In [ ]:
bgm = nltk.collocations.BigramAssocMeasures()

In [ ]:
finder = nltk.collocations.BigramCollocationFinder.from_words(no_stopwords)

In [ ]:
scored = finder.score_ngrams(bgm.likelihood_ratio)

In [ ]:
prefix_keys = collections.defaultdict(list)
for key, scores in scored:
   prefix_keys[key[0]].append((key[1], scores))

In [ ]:
# Sort keyed bigrams by strongest association.                                  
for key in prefix_keys:
   prefix_keys[key].sort(key = lambda x: -x[1])

print('share',prefix_keys['share'][:15])
print("----------")
print('repurchase', prefix_keys['repurchase'][:15])
print("-----------")
print('authorization', prefix_keys['authorization'][:15])
print("------------")
print('directors', prefix_keys['directors'][:15])

# trigram scoring using nltk collocations



In [ ]:
tgm = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_words(no_stopwords)
scored = finder.score_ngrams(tgm.likelihood_ratio)
prefix_keys = collections.defaultdict(list)
for key, scores in scored:
   prefix_keys[key[0]].append((key[1], scores))

# Sort keyed trigrams by strongest association.                                  
for key in prefix_keys:
   prefix_keys[key].sort(key = lambda x: -x[1])

print('share repurchase',prefix_keys['share'][:15])
print("----------")
print('repurchase date', prefix_keys['repurchase'][:15])
print("-----------")
print('authorization date', prefix_keys['authorization'][:15])
print("------------")
print('board directors', prefix_keys['directors'][:15])


In [ ]:
type(prefix_keys)

In [ ]:
print([key for key, value in prefix_keys.items()])
